In [33]:
import PIL
from PIL import Image
import numpy as np 
import requests
import json
import base64
# Function to Read and Convert Images
def load_resize_image(img):
    im = Image.open(img)
    image = im.resize((224,224)) 
    image = np.array(image)
    return image

In [34]:
img='/home/mateo/code/Matbeac/280285_2200-732x549.jpg'

In [35]:
img=load_resize_image(img)

In [36]:
t=img.reshape(img.shape[0]*img.shape[1]*img.shape[2]).astype(np.uint8)
s=base64.b64encode(t)

In [37]:
response=base64.b64decode(s)
q=np.frombuffer(s,dtype=np.uint8)
response_reshape=q.reshape((height,width,color))

NameError: name 'height' is not defined

# Trying image to text

In [27]:
import re

In [29]:
output_string=""""LONDON
HOUSE

A LA CARTE MENU

Marinated Nocellara olives 5
Harissa spiced nuts 3.5

Bread basket, olive oil & aged balsamic 4.5

STARTERS

Burrata, heirloom tomato, basil & toasted sourdough 11.5
Chicken Caesar salad, anchovies, croutons 12/17.5
Hiramasa kingfish crudo, avocado, chili & ginger dressing 13.5

Chicken & duck liver parfait, brioche & grape chutney L6

MAINS
Harissa roasted cauliflower, warm chickpea salad, chimichurri 20.5
Grilled hake, romesco sauce, fennel & piquillo pepper 24
Parsley & basil chicken Kiev, creamed potatoes, salsa verde 18
Steak frites, roasted beef tomato, watercress 27
BBQ lamb, labneh, fennel and chili slaw 26.5

Fish & chips, tartare sauce, curry sauce, mushy peas 19.5

SIDES

Koffmann's fries 5.5

Heirloom tomato & basil salad 5.5

Seasonal greens & sage dressing 5.5

SWEETS
Chocolate brownic, dulce de leche, caramel sundae 9
Cinnamon doughnuts, passion [ruit curd 7
Sticky toffee pudding & vanilla ice cream 9

Mango sorbet, marinated berries 7.5

 

If you have a food allergy, intolerance or sensitivity, please speak lo your sereer

about ingredients in our dishes before you order your meal.

   

An optional 15% harge will be added to your All prices are inclusive of VAT.

 

ere

   

rdsLondonTTouseGordonRamsay Eiiondonhousegordonramsay


Envoyer un message à Nuria Aires



















"""""

In [31]:
ingredient_words=re.split('\s+', output_string)

In [32]:
ingredient_words

['"LONDON',
 'HOUSE',
 'A',
 'LA',
 'CARTE',
 'MENU',
 'Marinated',
 'Nocellara',
 'olives',
 '5',
 'Harissa',
 'spiced',
 'nuts',
 '3.5',
 'Bread',
 'basket,',
 'olive',
 'oil',
 '&',
 'aged',
 'balsamic',
 '4.5',
 'STARTERS',
 'Burrata,',
 'heirloom',
 'tomato,',
 'basil',
 '&',
 'toasted',
 'sourdough',
 '11.5',
 'Chicken',
 'Caesar',
 'salad,',
 'anchovies,',
 'croutons',
 '12/17.5',
 'Hiramasa',
 'kingfish',
 'crudo,',
 'avocado,',
 'chili',
 '&',
 'ginger',
 'dressing',
 '13.5',
 'Chicken',
 '&',
 'duck',
 'liver',
 'parfait,',
 'brioche',
 '&',
 'grape',
 'chutney',
 'L6',
 'MAINS',
 'Harissa',
 'roasted',
 'cauliflower,',
 'warm',
 'chickpea',
 'salad,',
 'chimichurri',
 '20.5',
 'Grilled',
 'hake,',
 'romesco',
 'sauce,',
 'fennel',
 '&',
 'piquillo',
 'pepper',
 '24',
 'Parsley',
 '&',
 'basil',
 'chicken',
 'Kiev,',
 'creamed',
 'potatoes,',
 'salsa',
 'verde',
 '18',
 'Steak',
 'frites,',
 'roasted',
 'beef',
 'tomato,',
 'watercress',
 '27',
 'BBQ',
 'lamb,',
 'labneh,',
 

In [ ]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[]}
    missing_ingredients={'missing_ingredients':[]}
    for ingredient in output_df['ingredient']:
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
    final_df=pd.DataFrame(final_dict)
    
    
    # Adding the columns "value" and metric from output_dict
    final_df=final_df.merge(output_df,on="ingredient")

    return final_df,missing_ingredients